# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
# load data from database
engine = create_engine('sqlite:///ElliottPipeline.db')
conn = engine.connect()
df = pd.read_sql_table('messagesCategorized', conn)
X = df.drop(['genre'], axis = 1)
Y = df['genre'] 
X.head()

,id,message,original,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
Y.head()

0    direct
1    direct
2    direct
3    direct
4    direct
Name: genre, dtype: object

In [4]:
Y.value_counts()

news      13039
direct    10747
social     2394
Name: genre, dtype: int64

### 2. Write a tokenization function to process your text data

In [6]:
import nltk
import re
nltk.download('punkt')

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokenized = nltk.word_tokenize(text)
    return tokenized

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
tokenize(X.message[438])

['we',
 'are',
 'at',
 'the',
 'church',
 'of',
 'l',
 'eglise',
 'evangelique',
 'of',
 'claude',
 'noel',
 'we',
 'are',
 'inside',
 'please',
 'bring',
 'food',
 'and',
 'water',
 'for',
 'us']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import GradientBoostingClassifier

In [40]:
# featureU = FeatureUnion('TextEngPipe', Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer())
#         ]))

pipeline = Pipeline([
    ('feats', FeatureUnion([
        ('TextEngPipe', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ]))
    ])),

        ('gbc', GradientBoostingClassifier())
])



#      ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ]))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
Y2 = pd.get_dummies(Y)

In [13]:
X2 = X['message']

In [36]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y)

In [37]:
Xtrain.shape[0] == Ytrain.shape[0]

True

In [38]:
Xtest.shape[0] == Ytest.shape[0]

True

In [39]:
pipeline.fit(Xtrain, Ytrain)

ValueError: Found input variables with inconsistent numbers of samples: [39, 19635]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [134]:
yhat = pipeline.predict(Xtest)

from sklearn.metrics import classification_report

print(classification_report(Ytest, yhat))

             precision    recall  f1-score   support

     direct       0.92      0.91      0.91      2710
       news       0.92      0.95      0.94      3237
     social       0.96      0.87      0.92       598

avg / total       0.93      0.93      0.93      6545



In [135]:
confusion_matrix(Ytest, yhat)

array([[2462,  230,   18],
       [ 159, 3074,    4],
       [  52,   23,  523]])

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.